######  <span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#c3b235">NLP text preprocessing

- __In NLP text preprocessing is more essential, and NLP is an art to extract some information from the text documents.__
- ___First step is the preprocessing, and how to process the data are given in detail___



- __Import Libraries and download NLTK tools__

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
from termcolor import colored

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stop')
nltk.download('stopwords')
# nltk.download('all')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jayanthikishore/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jayanthikishore/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/jayanthikishore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading stop: Package 'stop' not found in index
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jayanthikishore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#ea0ea1">Text datasets reading

In [2]:
tdata1 = pd.read_csv("~/Downloads/ML_classwork/Week7_srrt/tweets_labels.csv")

In [3]:
tdata1

,*screams in 25 different languages*,0.6
0,Families to sue over Legionnaires: More than 4...,0.1
1,Pandemonium In Aba As Woman Delivers Baby With...,0.4
2,My emotions are a train wreck. My body is a tr...,0.2
3,Alton brown just did a livestream and he burne...,0.5
4,@TinyJecht Are you another Stand-user? If you ...,0.5
...,...,...
1858,@Trollkrattos Juan Carlos Salvador The Secret ...,0.5
1859,@devon_breneman hopefully it doesn't electrocu...,0.5
1860,Businesses are deluged with invokces. Make you...,0.5
1861,#BREAKING411 4 police officers arrested for ab...,0.1


######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#830eea">Text dataset information and shape

In [4]:
#Total information of the data file
tdata1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1863 entries, 0 to 1862
Data columns (total 2 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   *screams in 25 different languages*  1863 non-null   object 
 1   0.6                                  1863 non-null   float64
dtypes: float64(1), object(1)
memory usage: 29.2+ KB


In [5]:
#Data shape
tdata1.shape

(1863, 2)

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#330eea">Rename column name

In [6]:
# renamming the column names for the text dataset
tdata = tdata1.rename(columns={"*screams in 25 different languages*":"tweet", "0.6":"lbel"})
tdata.tweet

0       Families to sue over Legionnaires: More than 4...
1       Pandemonium In Aba As Woman Delivers Baby With...
2       My emotions are a train wreck. My body is a tr...
3       Alton brown just did a livestream and he burne...
4       @TinyJecht Are you another Stand-user? If you ...
                              ...                        
1858    @Trollkrattos Juan Carlos Salvador The Secret ...
1859    @devon_breneman hopefully it doesn't electrocu...
1860    Businesses are deluged with invokces. Make you...
1861    #BREAKING411 4 police officers arrested for ab...
1862    @News@ Refugio oil spill may have been costlie...
Name: tweet, Length: 1863, dtype: object

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e4cea">Display the first document from the text document

In [7]:
#Displaying the first document
tdata.tweet[0]

"Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/ZA4AXFJSVB"

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0e4cea">Binning the y column

In [8]:
# Binning the y values to integers
bins = np.array([0.0,0.25,0.5,0.75])
# bins = np.array([0.0,0.2,0.4,0.6,0.8])
tdata['label'] = np.digitize(tdata['lbel'],bins)
# tdata['label'] = np.digitize(tdata['lble'],bins=[0.5])
tdata.head(5)

,tweet,lbel,label
0,Families to sue over Legionnaires: More than 4...,0.1,1
1,Pandemonium In Aba As Woman Delivers Baby With...,0.4,2
2,My emotions are a train wreck. My body is a tr...,0.2,1
3,Alton brown just did a livestream and he burne...,0.5,3
4,@TinyJecht Are you another Stand-user? If you ...,0.5,3


In [9]:
ptext = tdata.tweet
ydata = tdata.label
ptext.shape

(1863,)

In [10]:
print(ptext[0])

Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/ZA4AXFJSVB


######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0eead0">Text preprocessing

In [11]:
import re
import string
# temp = []
snow = nltk.stem.SnowballStemmer('english')
def preprocess(data):
    temp = []
    for sentence in data:
        sentence = sentence.lower()     #converting lowercase

        #Removal of HTTP links/URLs mixed up in any text:
        sentence = re.sub('http://\S+|https://\S+', '', sentence)
        #OR 
        #sentence = re.sub('http[s]?://\S+', '', sentence)
        #sentence = re.sub(r'https?:\/\/.*[\r\n]*','',sentence, flags=re.MULTILINE) #remove the URLs

        #punctuations:'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        sentence = "".join([char for char in sentence if char not in string.punctuation])
        sentence = re.sub(r'\d+', '', sentence)    # removing numbers
        sentence = sentence.strip()        # removing extra white spaces
        tokens = nltk.word_tokenize(sentence)[2 :]
    #     tokens = nltk.word_tokenize(sentence)
        sentence = [l.lower() for l in tokens]
        filtered_result = list(filter(lambda l: l not in stop_words, sentence))
        lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]    

        #(or)
    #     sentence = re.sub(r'[?|!|\'|"|#!$%&()*+-@]',r'',sentence)
    #     sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)
        
        temp.append(lemmas) 
    return temp

In [12]:
# Xdata_cleaned = preprocess(ptext)
# Xtest_cleaned = preprocess(Xtest)
ptext_cleaned = preprocess(ptext)

In [13]:
# print(Xdata_cleaned[0])
ptext_cleaned[:2]

[['sue',
  'legionnaire',
  'family',
  'affected',
  'fatal',
  'outbreak',
  'legionnaire',
  'disea'],
 ['aba', 'woman', 'delivers', 'baby', 'without', 'face', 'photo']]

In [14]:
# Normalized or cleaning dataset adding into original datafile for chcking
tdata['Normalized_tweet'] = ptext_cleaned
tdata.shape

(1863, 4)

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0eeaa4">Original text and cleaned/normalized text

In [15]:
tdata[['tweet','Normalized_tweet']].head(5)

,tweet,Normalized_tweet
0,Families to sue over Legionnaires: More than 4...,"[sue, legionnaire, family, affected, fatal, ou..."
1,Pandemonium In Aba As Woman Delivers Baby With...,"[aba, woman, delivers, baby, without, face, ph..."
2,My emotions are a train wreck. My body is a tr...,"[train, wreck, body, train, wreck, im, wreck]"
3,Alton brown just did a livestream and he burne...,"[livestream, burned, butter, touched, hot, pla..."
4,@TinyJecht Are you another Stand-user? If you ...,"[another, standuser, detonate, killer, queen]"


######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0eea7c">Bag of Words

In [16]:
from nltk import word_tokenize
word2count = {}
for data in ptext_cleaned:
    words = data 
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
            
word2count

{'sue': 5,
 'legionnaire': 12,
 'family': 24,
 'affected': 7,
 'fatal': 11,
 'outbreak': 6,
 'disea': 5,
 'aba': 3,
 'woman': 11,
 'delivers': 2,
 'baby': 7,
 'without': 15,
 'face': 14,
 'photo': 11,
 'train': 26,
 'wreck': 14,
 'body': 34,
 'im': 47,
 'livestream': 1,
 'burned': 9,
 'butter': 4,
 'touched': 1,
 'hot': 9,
 'plate': 2,
 'soon': 2,
 'made': 10,
 'nut': 1,
 'joke': 3,
 'another': 12,
 'standuser': 2,
 'detonate': 9,
 'killer': 3,
 'queen': 3,
 'timed': 1,
 'concert': 2,
 'screamed': 11,
 'minute': 17,
 'straight': 2,
 'florida': 3,
 'forest': 18,
 'service': 19,
 'firefighter': 2,
 'could': 17,
 'deployed': 1,
 'california': 24,
 'help': 13,
 'contain': 2,
 'fire': 73,
 'detail': 1,
 'atomic': 9,
 'bomb': 22,
 'japan': 8,
 'still': 32,
 'struggle': 3,
 'war': 20,
 'past': 6,
 'anniversary': 6,
 'devastation': 9,
 'wrought': 4,
 'b': 6,
 'killed': 20,
 'civilian': 3,
 'ground': 4,
 'jet': 3,
 'first': 24,
 'bombed': 10,
 'city': 13,
 'main': 3,
 'street': 8,
 'dramaticall

In [17]:
print("Word count :" ,colored(len(word2count),'blue'))

Word count : 5830


In [18]:
# selecting best 100 features only
import heapq 
ptext_words = heapq.nlargest(1200, word2count, key=word2count.get)

######  <span style="font-family: Arial; font-weight:bold;font-size:1.em;color:#0eea3a">Vectorization
    
    *Arbitarary words (units/tokens) into fixed length of vectors

In [19]:
# Converting sentences into vectors: Xtrain data
x_final = []
for data in ptext_cleaned:
    vec = []
    for word in ptext_words:
        if word in data: 
            vec.append(1)
        else:
            vec.append(0)
    x_final.append(vec)
    
X_final = np.array(x_final)
print(X_final)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
